# DataPrep

Data Preparation é o processo de limpeza e transformação de dados “brutos” antes de realizar análises e criar modelos. 

## 1. Tratamento de Missings
Muito comum na fase de EDA (Exploratory Data Analysis), precisamos tratar **dados faltantes** no dataset.

As principais razões para termos dados faltantes (_missing values_) são:


- **Missing at Random (MAR):** A propensão para um dado estar faltando não está relacionado à falta do dado em si, mas sim a algum dado observado. A probabilidade de o dado ser faltante é a mesma dentro de cada subgrupo de **dados observados**.
    - Exemplo: Estimar a renda bruta anual de uma residência em uma população, a partir de questionários. No caso do MAR, a falta de dados é aleatória em subgrupos de outras variáveis observadas. Por exemplo, suponha que você coletou dados da profissão de cada indivíduo no questionário, e deduziu que gerentes, VIPs, etc. são mais propensos a não compartilhar suas receitas. Então, dentro desses subgrupos de profissões, a falta de dados é aleatória.
    - Essa hipótese é mais geral e realista que o MCAR. Os métodos para lidar com missing data atuais normalmente partem da hipótese do MAR.

- **Missing Completely at Random (MCAR):** A probabilidade de o dado ser faltante é a mesma em todos os casos, e não estão relacionadas aos dados em si.
- Exemplo: Estimar a renda bruta anual de uma residência em uma população, a partir de questionários. No caso do MCAR, a falta de dados é completamente aleatória, como se alguns questionários fossem perdidos.

- **Missing Not at Random (MNAR):** Os valores faltantes dependem de outras variáveis, e a probabilidade de os valores estarem faltando são desconhecidas. Em geral, são situações difíceis de reconhecer e tratar.
- Exemplo: Pessoas não querem revelar suas rendas, porque elas têm vergonha dela.
- Estratégias para achar dados sobre as causas de falta são: encontrar mais dados sobre as causas de falta, fazer análise what-if para ver como os dados respondem em vários cenários.
- Possibilidade de se escalar a coleta, produzirmos mais dados faltantes (já que depende de variáveis do problema).


Nos casos de **falta aleatória**, podemos remover os dados com segurança de não introduzir bias no modelo. Já no caso de **falta não aleatória**, não podemos simplesmente remover os dados faltantes, pois eles foram introduzidos sistematicamente, e a sua remoção pode fazer o modelo se comportar de forma inadequada em relação à realidade (pode gerar bias).

Estas são algumas estratégias comumente adotadas para lidar com missing values:

![handling_missing_data_summary](img/handling_missing_data_summary.png)

### 1.1 Listwise
Remove todos os dados para um observação que tenha **pelo menos um missing value.**

- Aplicável se poucas observações têm missing values.
- Desvantajoso na maioria dos casos, pois a suposição de MCAR raramente é válida. Logo, a simples remoção dos missing values pode levar a estimações enviesadas.

Em Python:

`mydata.dropna(inplace=True)`


### 1.2 Pairwise
Analisa os exemplos que têm valores não-nulos para as variáveis de interesse, no momento em que se deseja fazer um determinado cálculo.

Veja o exemplo da imagem abaixo, em que se deseja calcular uma matriz de correlações:

![corr_matrix](img/corr_matrix.png)

Problema:
- Cada correlação na matriz do caso exposto vai ser calculada usando uma quantidade diferente de exemplos.
- Assume que os dados faltantes são MCAR

### 1.3 Dropping Variables
Remover colunas que ultrapassem um threshold de missing values (p. ex.: >= 60%).

Em Python:

`del mydata.column_name`

ou

`mydata.drop('column_name', axis=1, inplace=True)`

- Em geral, métodos de imputação costumam ser melhores do que métodos de remoção de features.


### 1.4 Time-Series methods

#### 1.4.1 Last Observation Carried Forward (LOCF) & Next Observation Carried Backward (NOCB)

- LOCF: O último ponto registrado da série (o mais recente) é atribuído ao ponto faltante.

- NOCB: O próximo ponto registrado da série é atribuído ao ponto faltante.

Ex.:

series = 3, 4, 5, 6, NA, 7, 8

LOCF(series) = 3, 4, 5, 6, **6**, 7, 8

NOCB(series) = 3, 4, 5, 6, **7**, 7, 8

#### 1.4.2 Linear Interpolation
Faz uma interpolação linear na variável com dados faltantes.

Problema:
- Pode não funcionar bem com dados sazonais, pois vai distorcer o comportamento habitual dos dados em períodos com características próprias.

#### 1.4.3 Seasonal Adjustement + Linear Interpolation

Tenta alinhar tendências de temporada com a interpolação.


### 1.5 Média, Mediana, Moda

Imputar a média, a mediana ou a moda do campo com nulos.

- Desvantagem: Pode reduzir a variância do dataset

### 1.6 Por regressão linear

Utilizamos um campo com valor faltando e usamos como target, para treinar uma regressão linear.

Onde tem missing values, colocamos no dataset de teste. Onde tem valor para a target, usamos como treino.

Assim, conseguimos aproximar o valor dos missing values por um modelo.

- Desvantagens:
- Overfitting (os valores aprendidos utilizaram as features do dataset, pode levar a resultados ruins quando utilizamos a feature “aprendida” em um outro modelo).
- Supõe relação linear com a feature alvo.


### 1.7 Multiple imputation

Consistem em imputar o dataset com missings $m$ vezes, com valores provenientes de uma distribuição. Utiliza-se a abordagem da simulação de Markov Chain Monte Carlo (MCMC). Assim, obtemos $m$ datasets completos, os quais reunimos em um só, fazendo a média dos missings imputados de cada dataset, por exemplo.

![multiple_imputation](img/multiple_imputation.png)

## 2. Tratamento de Outliers
**Outliers** são valores extremos que destoam do restante dos dados.

### 2.1 Tipos de Outliers
Por quantidade de features envolvidas:
- Univariados (relativos a uma única feature)
- Multivariados (relativos a mais de uma feature)

Por ambiente:
- Point outliers: pontos isolados que se encontram longe do restante da distribuição
- Contextual outliers: ruído nos dados, como símbolos de pontuação em análises de texto ou barulho de fundo em reconhecimento de voz
- Collective outliers: pontos que representam novidades 

### 2.2 Causas
As principais causas para a ocorrência de outliers são:

- Erros de entrada de dados (erros humanos)
- Erros de medida (erros instrumentais)
- Erros experimentais (extração de dados)
- Intencionais (dummy outliers para testar métodos de detecção)
- Erros de processamento de dados (manipulação de dados)
- Erros de amostragem (extração ou mistura de dados de fontes erradas ou diversas)
- Naturais (não são erros, apenas novidades)


### 2.3 Detecção de outliers
### Z-Score
O z-score de uma observação é uma métrica que indica a quantos desvios padrões um ponto está da média da amostra (assumindo uma distribuição gaussiana): $z = \frac{x - \mu}{\sigma}$. Valores comumente utilizados são:  2.5, 3 ou 3.5.

OBS: frequentemente os dados não são descritos por uma distribuição gaussiana. Esse problema pode ser solucionado aplicando-se algum tipo de transformação nos dados (ex: scaling).

#### DBScan
O algoritmo de clusterização DBScan pode ser usado para detectar outliers (pontos que não se encontram na vizinhança de raio $eps$ de um **core point**).


#### Isolation Forests
O princípio básico das Isolation Forests é o de que outliers são **poucos e distantes do restante das observações**.

Para isolar os outliers, uma feature é selecionada do conjunto de features aleatoriamente. Em seguida, é feita uma partição aleatória entre o valor máximo e mínimo para aquela feature. Através dessa partição aleatória, é possível detectar os outliers, que provavelmente estarão mais próximos da raiz da árvore (terão um “path length” menor do que os outros).

- Aplicado para problema de **detecção de anomalias**
- Vantagem em relação à construção de um perfil: ao invés de tentar construir um modelo de instâncias normais, esse método explicitamente isola pontos anômalos no dataset.

![isolation_forest](img/isolation_forest.png)

## 3. Categorização de Variáveis Contínuas e Discretas
### 3.1 Label Encoding
Se as variáveis são ordenadas (ex: "best" > "good" > "fair" > "bad") uma boa solução para transformá-las em categorias é simplesmente substituí-las por números. Ex: best = 2, good = 1, fair = 0, bad = -1.

### 3.2 Dummy Encoding (One-Hot)
Criar uma nova feature para cada valor que a variável pode assumir.

#### Quando usar
- Problemas com poucos “níveis”
- Valores da variável não ordenados (ex.: nomes de países, nomes de estados, ...)
- Modelos lineares (com qualquer quantidade de níveis)
- Problemas em que o número de níveis é conhecido (é pouco provável surgirem novos)

#### Quando não usar
Modelos de árvore não lidam bem com one-hot-encodings de muitos níveis. Geralmente, essas features acabam sendo ignoradas no processo de construção da árvore.


#### Dummy Trap
**Excluir uma das colunas geradas pelo One-Hot-Encoding.**
O argumento para isso é estatístico: se não excluirmos uma das colunas, a soma dessas colunas será 1 para todas as linhas. Isso impacta a interpretabilidade dos coeficientes.

Ex: modelo que fornece o custo de se alimentar determinado animal.

$cost = 10*Species_{Cat} + 15*Species_{Dog} + 8*Species_{Rabbit} + 5*Species_{Snake}$

Podemos escrever isso de outra forma, começando com um custo = 6 e reduzindo os valores dos coeficientes, como mostrado abaixo:

$cost = 6 + 4*Species_{Cat} + 9*Species_{Dog} + 2*Species_{Rabbit} + (-1)*Species_{Snake}$

Se tiramos um dos animais (ex: gato), o valor do intercept terá que ser obrigatoriamente 10, o que acaba fixando os outros coeficientes também:

$cost = 10 + 5*Species_{Dog} + (-2)*Species_{Rabbit} + (-5)*Species_{Snake}$

Na prática, muitas pessoas ignoram a dummy trap pois:
- Introduz um viés em relação à variável excluída durante a regularização. Olhando no exemplo anterior, o custo relativo a gatos foi incorporado pelo intercept, que não sofre regularização (enquanto os coeficientes dos outros animais, sim).
- Dificulta o tratamento de novos níveis. 

OBS: não usar pd.get_dummies.


## 4. PCA
Quando lidamos com um grande número de variáveis correlacionadas, os **componentes principais** permitem resumir esses dados a partir de um menor número de variáveis representativas, que coletivamente conseguem explicar a maior parte da variabilidade do dataset original. 

As direções dos componentes principais são aquelas ao longo das quais os dados originais apresentam **grande variabilidade**.

**PCA** (Principal Component Analysis) é o processo que permite computar os **componentes principais**. Em outras palavras, o PCA permite encontrar uma representação de menor dimensionalidade do dataset que é capaz de explicar a maior parte da variabilidade original.  Trata-se de um método **não-supervisionado**, uma vez que envolve apenas as features $X_1, …, X_n$, e não uma variável resposta $Y$.


### 4.1 Principal components
Cada um dos componentes principais é uma combinação linear de $p$ features. 

#### Primeiro componente
O primeiro componente principal de um conjunto de features $X_1, X_2, …, X_p$ é a combinação linear normalizada dessas features que contém a maior variância.

Podemos escrever esse componente como:

$Z_1 = \phi_{11}X_1, \phi_{21}X_2, …, \phi_{p1}X_p$

Onde $\sum_{j=1}^p\phi_{j1}^2 = 1$. Os elementos $\phi_{11}, .., \phi_{p1}$ são conhecidos como **loadings** do primeiro componente principal, e $\phi_1 = (\phi_{11}, .., \phi_{p1})^T$ é o **loading vector**. 
O loading vector do primeiro componente pode ser interpretado como sendo a linha em um espaço $p$-dimensional que é mais próxima das $n$ observações.

Considerando que temos $n$ observações, o dataset $X$ será uma matriz $n x p$. Como o único aspecto que interessa em PCA é a variância, as variáveis são centralizadas de forma a terem média 0. Em seguida, é calculado:

$z_{i1} = \phi_{11}x_{i1}, \phi_{21}x_{i2}, …, \phi_{p1}x_{ip}$

O problema de otimização do primeiro componente principal pode então ser escrito como:

$\text{maximize}_{ \phi_{11}, ... ,  \phi_{p1}} \{ \frac{1}{N}\sum_{i=1}^n (\sum_{j=1}^p \phi_{j1}x_{ij})^2 \} = \text{maximize}_{ \phi_{11}, ... ,  \phi_{p1}} \{ \frac{1}{N}\sum_{i=1}^n z_{i1}^2 \}$, sujeito a $\sum_{j=1}^p\phi_{j1}^2 = 1$.

$z_{11}, …, z_{n1}$ são chamados de **scores** do primeiro componente principal.

#### Segundo componente
O segundo componente é a combinação linear de $X_1, …, X_p$ que tem a maior variância entre todas as combinações lineares **não-correlacionadas com o primeiro componente principal**. Sendo $Z_1$ o primeiro componente principal e $Z_2$ o segundo, tem-se que $\phi_1$ é ortogonal a $\phi_2$


### 4.2 Visualizando PCA
Na figura abaixo, foi utilizado PCA no dataset de aprisionamentos nos EUA. Para cada um dos estados, o dataset possui a taxa (a cada 100 mil habitantes) para 3 tipos de crime (Assault, Murder e Rape), bem como a população UrbanPop.

O plot abaixo, chamado de **biplot** mostra os scores dos componentes principais (em azul) e os loading vectors (em laranja). 

Ex: o loading para Rape no primeiro componente é 0.54 e no segundo componente é 0.17. Observa-se que o primeiro componente atribui aproximadamente o mesmo peso para Rape, Assault e Murder, que é muito maior do que o peso atribuído para UrbanPop. Esse componente pode ser interpretado como um resumo das taxas de crimes. 

Já o segundo componente atribui um peso maior a UrbanPop que aos crimes, podendo ser interpretado como uma taxa de urbanização do estado.

![pca_components](img/pca_components.png)

Um outro exemplo é mostrado abaixo. Nesse caso, o dataset é tridimensional.

![pca_components_3d](img/pca_components_3d.png)

### 4.3 Outros detalhes
#### Scaling
Antes de realizar PCA, é importante **centralizar os dados** para que eles tenham média zero. Além disso, os resultados obtidos com PCA irão depender se as variáveis foram individualmente redimensionadas (cada uma multiplicada por uma constante diferente).

![pca_normalization](img/pca_normalization.png)

O mais recomendado é redimensionar cada variável de forma que todas tenham um **desvio padrão unitário**.

#### Visualização de dados
O método PCA também é util para a visualização de dados, pois permite reduzir os dados para dimensões representaveis visualmente (2D ou 3D). O unico problema, é que as features no novo espaço não têm um significado tão claro, visto que são combinações lineares das variaveis do problema no espaço original.

## 5. Correlação / Associação entre Dados Contínuos e entre Dados Discretos

### 5.1 Covariância
Covariância é a medida do quanto duas variáveis aleatórias mudam juntas.
Ex: Peso e altura de girafas possuem covariância positiva, pois quando uma dessas variáveis é grande, a outra tende a ser grande também.

Sejam $X$ e $Y$ duas variáveis aleatórias com médias $\mu_X$ e $\mu_Y$. A covariância de $X$ e $Y$ é definida como $cov(X,Y) = E[(X-\mu_X)(Y-\mu_Y)] = E[XY] - \mu_X\mu_Y$.
- $cov(X,X) = var(X)$,
- Se $X$ e $Y$ são independentes, $cov(X,Y) = 0$.

Ex: Jogamos uma moeda justa 3 vezes. Seja $X$ o número de caras nas primeiras duas jogadas, e $Y$ o número de caras nas últimas 2 jogadas. Computar $cov(X,Y)$

Com 3 jogadas, temos 8 resultados possíveis:
HHH, HHT, HTH, HTT, THH, THT, TTH, TTT (H = cara, T = coroa)

| X     \      Y | 0         | 1              | 2         |
|----------------|-----------|----------------|-----------|
| 0              | TTT (1/8) | TTH (1/8)      | - (0)     |
| 1              | HTT (1/8) | HTH, THT (1/4) | THH (1/8) |
| 2              | - (0)     | HHT (1/8)      | HHH (1/8) |

$p(X=0) = p(X=2) = 1/4$

$p(X=1) = 1/2$

$E(X) = E(Y) = 0*¼ + 2*¼ + 1*½ = 1$

$cov(X, Y) = \frac{1}{8}*(0 - 1)(0 - 1) + \frac{1}{8}*(0 - 1)(1 - 1) + 0*(0 - 1)(2 - 1) +
\frac{1}{8}*(1-1)(0-1) + \frac{1}{4}*(1-1)(1-1) + \frac{1}{8}*(1-1)(2-1) +
0*(2-1)(0-1) + \frac{1}{8}*(2-1)(1-1) + \frac{1}{8}*(2-1)(2-1) = \frac{1}{4}$


### 5.2 Correlação
As unidades de covariância são **unidades de X vezes unidades de Y**, o que dificulta a comparação de covariâncias. A correlação é uma forma de unificar as escalas das covariâncias:

$cor(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}$

- $-1 \leq cor(X,Y) \leq 1$


## 6. Bases Desbalanceadas
O que fazer quando as bases estão desbalanceadas:
- Coletar mais dados
- Mudar a métrica
- Usar diferentes algoritmos

### 6.1 Ajuste de amostragem
#### Oversampling
Consiste em **aumentar o número de observações da classe minoritária**. Para isso, é realizada uma amostragem com reposição.

#### Undersampling
Consiste em **diminuir o número de observações da classe majoritária**. Para isso, são usadas todas as observações da classe minoritária mais um conjunto aleatório de observações da classe majoritária.
